In [ ]:
from IPython.display import Markdown, clear_output, display
import ipywidgets as ipy
from ipywidgets import Layout
import matplotlib.pyplot as plt
from protomol.orca.writing import (
    standard_optimization_from_smiles,
    standard_scan_from_smiles,
)
from protomol.rd.mol import intra_proton_transfer
from protomol.util.render import from_allxyz_block, from_xyz_block
from utils import smi, visualize, widgets

from pathlib import Path
import re
import sqlite3

# Link to results database & display combobox of SMILES
conn = sqlite3.connect(Path.home() / "C5O-Kinetics/db/results.db")
cursor = conn.cursor()
cursor.execute("SELECT * FROM calculations")
rows = cursor.fetchall()
smiles = sorted(list(set(row[1] for row in rows)))
smiles_inp = widgets.combobox(smiles, "SMILES: ")


# Update SMILES
def on_update_smiles(btn):
    clear_output()
    display(header)
    smiles = smiles_inp.value
    cursor.execute(f"SELECT * FROM calculations WHERE smiles = '{smiles}'")
    rows = cursor.fetchall()
    methods = [(f"{row[3]}: {row[2]}", row[5]) for row in rows]
    if len(methods) != 0:
        header[1, :2] = widgets.dropdown(methods, description="Calculations: ")
        header[1, 2] = widgets.button("Visualize XYZ")
        header[1, 2].on_click(on_visualize_xyz)
        header[1, 3] = delete_calc_btn
    else:
        header[1, 0] = widgets.int_text(description="Multiplicity: ")
        header[1, 1] = widgets.dropdown(
            options=[("XTB Goat > REVDSD OptFreq > CCSDT Opt", "from_smiles")],
            description="Procedure: ",
        )
        header[1, 2] = widgets.button("Initialize Files")
        header[1, 2].on_click(on_submit_optimization)
        visualize.visualize_smiles(smiles_inp.value)


update_smiles_btn = widgets.button(description="Update SMILES")
update_smiles_btn.on_click(on_update_smiles)


# Visualize XYZ
def on_visualize_xyz(btn):
    clear_output()
    display(header)
    if not header[1, :2].value == "N/A":
        if "Relaxed Surface Scan" in header[1, :2].value:
            from_allxyz_block(header[1, :2].value.replace(">\n", ""))
            matches = re.findall(r"Step (\d+) E ([-\d\.]+)", header[1, :2].value)
            xyzs = header[1, :2].value.split(">")
            options = [(f"Step {match[0]}", xyz) for match, xyz in zip(matches, xyzs)]
            header[2, :2] = widgets.dropdown(
                options=options,
                description="Steps: ",
            )
            header[2, 2] = submit_scan_opt_btn
            if matches:
                steps = [int(step) for step, energy in matches]
                energies = [float(energy) for step, energy in matches]
                fig, ax = plt.subplots()
                ax.plot(steps, energies, marker="o")
                ax.set_xlabel("Step")
                ax.set_ylabel("Energy")
                plt.show()
        else:
            from_xyz_block(header[1, :2].value, add_labels=True)
            header[2, :2] = widgets.dropdown(
                options=["Intramolecular Proton Transfer"],
                description="Mechanisms: ",
            )
            header[2, 2] = update_trans_btn


# Submit new optimization
def on_submit_optimization(btn):
    multiplicity = header[1, 0].value
    if "from_smiles" in header[1, 1].value:
        sh_cmd = standard_optimization_from_smiles(
            smiles=smiles_inp.value, multiplicity=multiplicity
        )
        clear_output()
        display(header, Markdown(f"{sh_cmd}\n"))


# Delete SMILES from database
def on_delete_smiles(btn):
    smiles = smiles_inp.value
    if not smiles:
        display(Markdown("**Please select a SMILES to delete.**"))
        return
    display(
        Markdown(f"**Warning:** You are about to delete all entries for `{smiles}`.")
    )
    confirm_btn = widgets.button(description="Confirm Delete", button_color="red")
    cancel_btn = widgets.button(description="Cancel")
    confirm_box = ipy.HBox([confirm_btn, cancel_btn])
    display(confirm_box)

    def on_confirm(b):
        cursor.execute("DELETE FROM calculations WHERE smiles = ?;", (smiles,))
        conn.commit()
        display(Markdown(f"Deleted all entries for `{smiles}`."))
        clear_output()
        display(header)

    def on_cancel(b):
        clear_output()
        display(header)

    confirm_btn.on_click(on_confirm)
    cancel_btn.on_click(on_cancel)


delete_smi_btn = widgets.button(description="Delete SMILES")
delete_smi_btn.on_click(on_delete_smiles)


delete_calc_btn = widgets.button(description="Delete calculation")
# Add on click for delete calc


# Update transition options
def on_update_trans(b):
    clear_output()
    display(header)
    mechanism = header[2, :2].value
    if mechanism == "Intramolecular Proton Transfer":
        indices = smi.map_atomic_symbols_to_indices(smiles_inp.value)
        header[3, 0] = widgets.dropdown(
            options=[t for t in indices if t[0].startswith("H")], description="Proton: "
        )
        header[3, 1] = widgets.dropdown(
            options=[t for t in indices if not t[0].startswith("H")],
            description="Acceptor: ",
        )
        header[3, 2] = submit_scan_btn
        display(visualize.visualize_smiles(smiles_inp.value))


update_trans_btn = widgets.button(description="Update transition")
update_trans_btn.on_click(on_update_trans)


def on_submit_scan(b):
    clear_output()
    display(header)
    mechanism = header[2, :2].value
    if mechanism == "Intramolecular Proton Transfer":
        xyz, _ = intra_proton_transfer(
            smiles_inp.value, header[3, 0].value, header[3, 1].value
        )
        display(from_xyz_block(xyz, add_labels=True))
        standard_scan_from_smiles(
            smiles_inp.value, header[3, 0].value, header[3, 1].value
        )


submit_scan_btn = widgets.button(description="Submit scan")
submit_scan_btn.on_click(on_submit_scan)


def on_submit_scan_opt(b):
    clear_output()
    display(header)
    xyz_block = header[2, :2].value
    cursor.execute(f"SELECT * FROM calculations WHERE xyz = '{header[1, :2].value}'")
    rows = cursor.fetchall()
    display(rows[0][2].split()[2:])
    # standard_optimization_from_scan(smiles=smiles_inp.value, xyz_block=xyz_block)


submit_scan_opt_btn = widgets.button(description="Submit scan optimization")
submit_scan_opt_btn.on_click(on_submit_scan_opt)

header = ipy.GridspecLayout(4, 4, layout=Layout(height="auto", width="auto"))
header[0, :2] = smiles_inp
header[0, 2] = update_smiles_btn
header[0, 3] = delete_smi_btn

display(header)

GridspecLayout(children=(Combobox(value='C[CH]CC1CO1', description='SMILES: ', layout=Layout(grid_area='widget…

'XTB SCAN 14 1'